<table>
<tr>                                                                                   
     <th>
         <div style='padding:15px;color:#030aa7;font-size:240%;text-align: center;font-style: italic;font-weight: bold;font-family: Georgia, serif'><a href="https://www.kaggle.com/datasets/arashnic/standardized-biomedical-images-medmnist">MedMNIST<br> Standardized Biomedical Images</a></div>
     </th>
     <th><img src="https://raw.githubusercontent.com/rbizoi/DeepLearning/refs/heads/main/images/medminstico.jpg" width="96"></th>
 </tr>
</table>




<table>
    <tr> 
        <th>
        <div style='text-align: center'>
        <img src="https://raw.githubusercontent.com/rbizoi/DeepLearning/refs/heads/main/images/medmnistv2.jpg" width="512">
        </div>
        </th>
        <th>
        <div style='text-align: center'>
        <img src="https://raw.githubusercontent.com/rbizoi/DeepLearning/refs/heads/main/images/medminstico.png" width="512">
        </div>
    </tr>
</table>

<div style='text-align: center'>
        <img src="https://raw.githubusercontent.com/rbizoi/DeepLearning/refs/heads/main/images/medminstdataset.png" width="1024">
</div>

<div style='padding:15px;color:#030aa7;font-size:100%;text-align: left;font-family: Georgia, serif'><a href="https://github.com/MedMNIST/MedMNIST/tree/main">Veuillez vous référer à la page Github officielle pour plus de détails.</a></div>

>> **site du projet** : https://medmnist.com/

>> **site de données**>> **site de données** : https://zenodo.org/records/10519652 : https://zenodo.org/records/10519652


# <b><div style='padding:18px;background-color:#d8dcd6;color:#030aa7;font-size:130%; border-radius:12px 12px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>Introduction</div></b>

In [ ]:
import os
# import json
# @param ["tensorflow", "jax", "torch"]
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['KERAS_BACKEND'] = 'tensorflow'  
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
# os.environ['TF_XLA_FLAGS']='--tf_xla_auto_jit=1,--tf_xla_always_defer_compilation=true'
# os.environ['XLA_FLAGS']='--xla_backend_optimization_level=0,--xla_gpu_autotune_level=4,--xla_gpu_disable_ptxas_optimizations=true,--xla_gpu_use_cudnn_batchnorm_level=2'
# os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ['TF_CPP_MIN_LOG_LEVEL']='5'

In [ ]:
! nvidia-smi

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>Import libriries </div></b>

In [ ]:
import numpy as np, pandas as pd, seaborn as sns, warnings, sys, pickle
from matplotlib import pyplot as plt
from datetime import datetime as dt

import matplotlib.font_manager as fm
from matplotlib.lines import Line2D

warnings.filterwarnings(action="ignore")

if int(str(sns.__version__).split('.')[1]) > 8 : 
    plt.style.use('seaborn-v0_8-darkgrid')
else:
    plt.style.use('seaborn-darkgrid')
    
sns.set(font_scale=2)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

tf.get_logger().setLevel('ERROR')# Suppress TensorFlow logging (2)

In [ ]:
print("Tensorflow\t : %s\tCUDA %s\tGPU %s\tXLA %s\nKeras\t\t : %s\nPandas\t\t : %s\nNumPy\t\t : %s"%
      (tf.__version__, 
       tf.test.is_built_with_cuda(), 
       tf.test.is_built_with_gpu_support(), 
       tf.test.is_built_with_xla(), 
       tf.keras.__version__, 
       pd.__version__, 
       np.__version__))

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>Initialisation les GPUs presents </div></b>

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

if len(physical_devices) >  0 :
    for i, gpu in enumerate(physical_devices):
        tf.config.experimental.set_memory_growth(physical_devices[i], True)

    strategy = tf.distribute.experimental.CentralStorageStrategy()
    print('Le système est initialisé avec {0:d} GPUs'.format(strategy.num_replicas_in_sync))

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>Outils du document </div></b>

In [ ]:
palette = [
            "#030aa7", "#e50000", "#d8863b", "#005f6a", "#6b7c85", "#751973", "#d1e5f0", "#fddbc7",
            "#ffffcb", "#12e193", "#d8dcd6", "#ffdaf0", "#dfc5fe", "#f5054f", "#a0450e",
            "#0339f8", "#f4320c", "#fec615", "#017a79", "#85a3b2", "#fe2f4a", "#a00498", "#b04e0f",
            "#0165fc", "#ff724c", "#fddc5c", "#11875d", "#89a0b0", "#fe828c", "#cb00f5", "#b75203",
            "#0485d1", "#ff7855", "#fbeeac", "#0cb577", "#95a3a6", "#ffb7ce", "#c071fe", "#ca6b02",
            "#92c5de", "#f4a582", "#fef69e", "#18d17b", "#c5c9c7", "#ffcfdc", "#caa0ff", "#cb7723",
            "#d1e5f0", "#fddbc7", "#ffffcb", "#12e193", "#d8dcd6", "#ffdaf0", "#dfc5fe", "#d8863b",
            "#030764", "#be0119", "#dbb40c", "#005249", "#3c4142", "#cb0162", "#5d1451", "#653700",
            "#040348", "#67001f", "#b27a01", "#002d04", "#000000", "#a0025c", "#490648", "#3c0008"
          ]
# sns.palplot(sns.color_palette(palette))

In [ ]:
nom_projet                = "MedMNIST-donnees"
repertoireProjet          = os.getcwd()
repertoireEnregistrement  = repertoireProjet +'/'+nom_projet+ '/repertoire.images'
repertoireSauvegardes     = repertoireProjet +'/'+nom_projet+ '/repertoire.sauvegardes'

def controleExistenceRepertoire(directory, create_if_needed=True):
    """Voir si le répertoire existe. S'il n'existe pas il est créé."""
    path_exists = os.path.exists(directory)
    if path_exists:
        if not os.path.isdir(directory):
            raise Exception("Trouvé le nom "+directory+" mais c'est un fichier, pas un répertoire")
            return False
        return True
    if create_if_needed:
        os.makedirs(directory)
        
controleExistenceRepertoire(repertoireEnregistrement)
controleExistenceRepertoire(repertoireSauvegardes)

def sauvegarderImage( fichier):
    """Enregistrez la figure. Appelez la méthode juste avant plt.show ()."""
    plt.savefig(os.path.join(repertoireEnregistrement,
                             fichier+f"--{dt.now().strftime('%Y_%m_%d_%H.%M.%S')}.png"), 
                             dpi=600, 
                             bbox_inches='tight')


    
def sauvegarderModelPoids(model, fichierPoids, repertoireSauvegardes=repertoireSauvegardes):
    """Enregistrez les poids du modèle Keras."""
    if fichierPoids != None:
        controleExistenceRepertoire(repertoireSauvegardes)
        nomFichier = os.path.join(repertoireSauvegardes, '{}.keras'.format(fichierPoids))
        model.save_weights(nomFichier)

def sauvegarderModel(model, fichier, repertoireSauvegardes=repertoireSauvegardes):
    """Enregistrez le modèle Keras."""
    if fichier != None:
        controleExistenceRepertoire(repertoireSauvegardes)
        nomFichier = os.path.join(repertoireSauvegardes, '{}.keras'.format(fichier))
        model.save(nomFichier)

def lectureModelPoids(model, fichier, repertoireSauvegardes=repertoireSauvegardes):
    """Si le fichier existe, il est chargé et retourne True, sinon retourne False."""
    nomFichier = os.path.join(repertoireSauvegardes, '{}.keras'.format(fichier))
    if os.path.exists(nomFichier):
        if os.path.isfile(nomFichier):
            model.load_weights(nomFichier)
            return True
    return False

def lectureModel(self, model_filename):
    """Si le fichier existe, il est chargé et retourne True, sinon retourne False."""
    fullpath = self.saved_models_dir+'/'+model_filename+'.keras'
    if os.path.exists(fullpath):
        if os.path.isfile(fullpath):
            model = load_model(fullpath)
            return model
    return None    

def sauvegardeHistorique(model,
                         repertoireSauvegardes,
                         nomSauvegarde='one_hidden_layer_history_batch_size_1'):

    history = pd.DataFrame( model.history)
    history.reset_index(inplace=True)
    history.rename(columns={'index':'epoch'},inplace=True)
    history.to_parquet(os.path.join(repertoireSauvegardes,f'{nomSauvegarde}.gzip'),compression='gzip', engine='pyarrow') 
    return history

def afficheHistoriqueEntrainement(history, palette):
    fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(48,16));
    markersize = 8
    linewidth=2
    
    graph = sns.lineplot(x='epoch', 
                         y='accuracy',  
                         data=history,
                         ax=ax[0],      
                         label='accuracy',
                         err_style=None, 
                         marker='o',
                         markersize=markersize,
                         linewidth=linewidth,
                         color=palette[0],
                         );
    graph = sns.lineplot(x='epoch', 
                         y='val_accuracy',  
                         data=history,
                         ax=ax[0],      
                         label='val_accuracy',
                         err_style=None, 
                         marker='o',
                         markersize=markersize,
                         linewidth=linewidth,
                         color=palette[1],
                         );
    
    graph = sns.lineplot(x='epoch', 
                         y='loss',  
                         data=history,
                         ax=ax[1],      
                         label='loss',
                         err_style=None, 
                         marker='o',
                         markersize=markersize,
                         linewidth=linewidth,
                         color=palette[0],
                         );
    graph = sns.lineplot(x='epoch', 
                         y='val_loss',  
                         data=history,
                         ax=ax[1],      
                         label='val_loss',
                         err_style=None, 
                         marker='o',
                         markersize=markersize,
                         linewidth=linewidth,
                         color=palette[1],
                         );
    sauvegarderImage('afficheHistoriqueEntrainement')
    

# <b><div style='padding:18px;background-color:#d8dcd6;color:#030aa7;font-size:130%; border-radius:12px 12px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>Lecture des données à partir du disque</div></b>

In [ ]:
def afficheDataset(donnees,labels,taille,image,dictLabels,cmap=None):
    plt.figure(figsize=(image, image))
    i=1
    for image, label in zip(donnees[:(taille*taille)],labels[:(taille*taille)]):
        ax = plt.subplot(taille, taille, i)
        if cmap is None :
            plt.imshow(image)
        else :
            plt.imshow(image,cmap='gray')
        plt.title(dictLabels[label[0]])
        plt.axis("off")
        i+=1

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>PathMNIST - RGB</div></b>

The **PathMNIST** is based on a prior study for predicting survival from colorectal cancer histology slides, providing a dataset (**NCT-CRC-HE-100K**) of **100,000** non-overlapping image patches from hematoxylin & eosin stained histological images, and a test dataset (**CRC-VAL-HE-7K**) of **7,180** image patches from a different clinical center.<br>
The dataset is comprised of **9** types of tissues, resulting in a multi-class classification task. <br>
We resize the source images of 3×224×224 into 3×28×28, and split **NCT-CRC-HE-100K** into training and validation set with a ratio of 9:1.<br> The **CRC-VAL-HE-7K** is treated as the test set.

>> https://zenodo.org/records/10519652/files/pathmnist.npz<br>
>> https://zenodo.org/records/10519652/files/pathmnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/pathmnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/pathmnist_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/pathmnist_64.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {0:"adipose",
              1:"background",
              2:"debris",
              3:"lymphocytes",
              4:"mucus",
              5:"smooth\nmuscle",
              6:"normal\ncolon mucosa",
              7:"cancer-associated\nstroma",
              8:"colorectal\nadenocarcinoma\nepithelium"}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels)

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>ChestMNIST - Grey</div></b>

The **ChestMNIST** is based on the **NIH-ChestXray14** dataset, a dataset comprising **112,120** frontal-view X-Ray images of **30,805** unique patients with the text-mined **14** disease labels, which could be formulized as a multi-label binary-class classification task.<br>
We use the official data split, and resize the source images of 1×1024×1024 into 1×28×28.

>> https://zenodo.org/records/10519652/files/chestmnist.npz<br>
>> https://zenodo.org/records/10519652/files/chestmnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/chestmnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/chestmnist_224.npz<br>


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/chestmnist.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0: "atelectasis",
1: "cardiomegaly",
2: "effusion",
3: "infiltration",
4: "mass",
5: "nodule",
6: "pneumonia",
7: "pneumothorax",
8: "consolidation",
9: "edema",
10: "emphysema",
11: "fibrosis",
12: "pleural",
13: "hernia"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels,cmap='gray')

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>DermaMNIST - RGB</div></b>

The **DermaMNIST** is based on the **HAM10000**, a large collection of multi-source dermatoscopic images of common pigmented skin lesions.<br>
The dataset consists of **10,015** dermatoscopic images categorized as 7 different diseases, formulized as a multi-class classification task. <br>
We split the images into training, validation and test set with a ratio of 7:1:2. The source images of 3×600×450 are resized into 3×28×28.

>> https://zenodo.org/records/10519652/files/dermamnist.npz<br>
>> https://zenodo.org/records/10519652/files/dermamnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/dermamnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/dermamnist_224.npz<br>


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/dermamnist_64.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0:"actinic keratoses and intraepithelial carcinoma",
1:"basal cell carcinoma",
2:"benign keratosis-like lesions",
3:"dermatofibroma",
4:"melanoma",
5:"melanocytic nevi",
6:"vascular lesions"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels)

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>OCTMNIST - Grey</div></b>

The **OCTMNIST** is based on a prior dataset of **109,309** valid optical coherence tomography (OCT) images for retinal diseases. The dataset is comprised of 4 diagnosis categories, leading to a multi-class classification task. We split the source training set with a ratio of 9:1 into training and validation set, and use its source validation set as the test set. The source images are gray-scale, and their sizes are (384−1,536)×(277−512). We center-crop the images and resize them into 1×28×28.

>> https://zenodo.org/records/10519652/files/octmnist.npz<br>
>> https://zenodo.org/records/10519652/files/octmnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/octmnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/octmnist_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/octmnist_64.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0: "choroidal\nneovascularization",
1: "diabetic\nmacular\nedema",
2: "drusen",
3: "normal"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels,cmap='gray')

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>PneumoniaMNIST - Grey</div></b>

The **PneumoniaMNIST** is based on a prior dataset of **5,856** pediatric chest X-Ray images. The task is binary-class classification of pneumonia against normal. We split the source training set with a ratio of 9:1 into training and validation set and use its source validation set as the test set. The source images are gray-scale, and their sizes are (384−2,916)×(127−2,713). We center-crop the images and resize them into 1×28×28.

>> https://zenodo.org/records/10519652/files/pneumoniamnist.npz<br>
>> https://zenodo.org/records/10519652/files/pneumoniamnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/pneumoniamnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/pneumoniamnist_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/pneumoniamnist.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0: "normal",
1: "pneumonia"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=20,image=64,dictLabels=dictLabels,cmap='gray')

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>RetinaMNIST - RGB</div></b>

The **RetinaMNIST** is based on the **DeepDRiD** challenge, which provides a dataset of **1,600** retina fundus images.<br> 
The task is ordinal regression for 5-level grading of **diabetic retinopathy severity**.<br> 
We split the source training set with a ratio of 9:1 into training and validation set, and use the source validation set as the test set.<br>
The source images of 3×1,736×1,824 are center-cropped and resized into 3×28×28.

>> https://zenodo.org/records/10519652/files/retinamnist.npz<br>
>> https://zenodo.org/records/10519652/files/retinamnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/retinamnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/retinamnist_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/retinamnist.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0:"0",
1:"1",
2:"2",
3:"3",
4:"4"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels)

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>BreastMNIST - Grey</div></b>

The **BreastMNIST** is based on a dataset of 780 breast ultrasound images.<br> 
It is categorized into 3 classes: normal, benign, and malignant.<br>
As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set.<br>
The source images of 1×500×500 are resized into 1×28×28.

>> https://zenodo.org/records/10519652/files/breastmnist.npz<br>
>> https://zenodo.org/records/10519652/files/breastmnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/breastmnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/breastmnist_224.npz

In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/breastmnist.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0: "malignant",
1: "normal, benign"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=20,image=64,dictLabels=dictLabels,cmap='gray')

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>BloodMNIST - RGB</div></b>

The **BloodMNIST** is based on a dataset of individual normal cells, captured from individuals without infection, hematologic or oncologic disease and free of any pharmacologic treatment at the moment of blood collection.<br>
It contains a total of **17,092** images and is organized into 8 classes. We split the source dataset with a ratio of 7:1:2 into training, validation and test set.<br>
The source images with resolution 3×360×363 pixels are center-cropped into 3×200×200, and then resized into 3×28×28.

>> https://zenodo.org/records/10519652/files/bloodmnist.npz<br>
>> https://zenodo.org/records/10519652/files/bloodmnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/bloodmnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/bloodmnist_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/bloodmnist_64.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {0:"basophil",
            1:"eosinophil",
            2:"erythroblast",
            3:"immature",# granulocytes(myelocytes, metamyelocytes and promyelocytes)",
            4:"lymphocyte",
            5:"monocyte",
            6:"neutrophil",
            7:"platelet"}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=56,dictLabels=dictLabels)

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>TissueMNIST - Gray</div></b>

We use the **BBBC051**, available from the Broad Bioimage Benchmark Collection.<br>
The dataset contains **236,386** human kidney cortex cells, segmented from 3 reference tissue specimens and organized into 8 categories.<br>
We split the source dataset with a ratio of 7:1:2 into training, validation and test set. <br>
Each gray-scale image is 32×32×7 pixels, where 7 denotes 7 slices. We take maximum values across the slices and resize them into 28×28 gray-scale images.

>> https://zenodo.org/records/10519652/files/tissuemnist.npz<br>
>> https://zenodo.org/records/10519652/files/tissuemnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/tissuemnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/tissuemnist_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/tissuemnist_64.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0:"Collecting Duct\nConnecting Tubule",
1:"Distal Convoluted Tubule",
2:"Glomerular\nendothelial cells",
3:"Interstitial\nendothelial cells",
4:"Leukocytes",
5:"Podocytes",
6:"Proximal\nTubule Segments",
7:"Thick\nAscending Limb"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels,cmap='gray')

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>OrganAMNIST - Gray</div></b>

The **OrganAMNIST** is based on **3D computed tomography (CT)** images from **Liver Tumor Segmentation Benchmark (LiTS)**.<br>
It is renamed from OrganMNIST_Axial (in MedMNIST v1) for simplicity.<br>
We use bounding-box annotations of 11 body organs from another study to obtain the organ labels.<br>
Hounsfield-Unit (HU) of the 3D images are transformed into gray-scale with an abdominal window.<br>
We crop 2D images from the center slices of the 3D bounding boxes in axial views (planes).<br>
The images are resized into 1×28×28 to perform multi-class classification of 11 body organs. 115 and 16 CT scans from the source training set are used as training and validation set, respectively.<br>
The 70 CT scans from the source test set are treated as the test set.

>> https://zenodo.org/records/10519652/files/organamnist.npz<br>
>> https://zenodo.org/records/10519652/files/organamnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/organamnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/organamnist_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/organamnist_224.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0 : "bladder",
1 : "femur-left",
2 : "femur-right",
3 : "heart",
4 : "kidney-left",
5 : "kidney-right",
6 : "liver",
7 : "lung-left",
8 : "lung-right",
9 : "pancreas",
10 : "spleen"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels,cmap='gray')

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>OrganCMNIST - Gray</div></b>

The **OrganAMNIST** is based on **3D computed tomography (CT)** images from **Liver Tumor Segmentation Benchmark (LiTS)**.<br>
It is renamed from OrganMNIST_Axial (in MedMNIST v1) for simplicity.<br>
We use bounding-box annotations of 11 body organs from another study to obtain the organ labels.<br>
Hounsfield-Unit (HU) of the 3D images are transformed into gray-scale with an abdominal window.<br>
We crop 2D images from the center slices of the 3D bounding boxes in axial views (planes).<br>
The images are resized into 1×28×28 to perform multi-class classification of 11 body organs. 115 and 16 CT scans from the source training set are used as training and validation set, respectively.<br>
The 70 CT scans from the source test set are treated as the test set.

>> https://zenodo.org/records/10519652/files/organcmnist.npz<br>
>> https://zenodo.org/records/10519652/files/organcmnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/organcmnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/organcmnist_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/organcmnist_64.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0 : "bladder",
1 : "femur-left",
2 : "femur-right",
3 : "heart",
4 : "kidney-left",
5 : "kidney-right",
6 : "liver",
7 : "lung-left",
8 : "lung-right",
9 : "pancreas",
10 : "spleen"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels,cmap='gray')

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>OrganSMNIST - Gray</div></b>

The **OrganAMNIST** is based on **3D computed tomography (CT)** images from **Liver Tumor Segmentation Benchmark (LiTS)**.<br>
It is renamed from OrganMNIST_Axial (in MedMNIST v1) for simplicity.<br>
We use bounding-box annotations of 11 body organs from another study to obtain the organ labels.<br>
Hounsfield-Unit (HU) of the 3D images are transformed into gray-scale with an abdominal window.<br>
We crop 2D images from the center slices of the 3D bounding boxes in axial views (planes).<br>
The images are resized into 1×28×28 to perform multi-class classification of 11 body organs. 115 and 16 CT scans from the source training set are used as training and validation set, respectively.<br>
The 70 CT scans from the source test set are treated as the test set.

>> https://zenodo.org/records/10519652/files/organsmnist.npz<br>
>> https://zenodo.org/records/10519652/files/organsmnist_64.npz<br>
>> https://zenodo.org/records/10519652/files/organsmnist_128.npz<br>
>> https://zenodo.org/records/10519652/files/organsmnist_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/organsmnist_64.npz")

In [ ]:
print(f'apprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0 : "bladder",
1 : "femur-left",
2 : "femur-right",
3 : "heart",
4 : "kidney-left",
5 : "kidney-right",
6 : "liver",
7 : "lung-left",
8 : "lung-right",
9 : "pancreas",
10 : "spleen"
}

In [ ]:
afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels,cmap='gray')

## <b><div style='padding:15px;background-color:#d8dcd6;color:#030aa7;font-size:100%; border-radius:10px 10px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>OrganMNIST3D - Gray</div></b>

The source of the **OrganMNIST3D** is the same as that of the **Organ{A,C,S}MNIST**.<br>
Instead of 2D images, we directly use the 3D bounding boxes and process the images into **28×28×28** to perform multi-class classification of 11 body organs.<br>
The same 115 and 16 CT scans as the **Organ{A,C,S}MNIST** from the source training set are used as training and validation set, respectively, and the same 70 CT scans as the **Organ{A,C,S}MNIST** from the source test set are treated as the test set.

>> https://zenodo.org/records/10519652/files/organmnist3d.npz<br>
>> https://zenodo.org/records/10519652/files/organmnist3d_64.npz<br>
>> https://zenodo.org/records/10519652/files/organmnist3d_128.npz<br>
>> https://zenodo.org/records/10519652/files/organmnist3d_224.npz


In [ ]:
donnees = np.load("../donnees/MedMNIST(Standardized Biomedical Image)/organmnist3d.npz")

In [ ]:
print(f'image size:\t{donnees["train_images"].shape}\napprentissage\t{donnees["train_images"].shape[0]}\nvalidation\t{donnees["val_images"].shape[0]}\ntest\t\t{donnees["test_images"].shape[0]}')

In [ ]:
dictLabels = {
0 : "bladder",
1 : "femur-left",
2 : "femur-right",
3 : "heart",
4 : "kidney-left",
5 : "kidney-right",
6 : "liver",
7 : "lung-left",
8 : "lung-right",
9 : "pancreas",
10 : "spleen"
}

afficheDataset(donnees["train_images"],donnees["train_labels"],taille=10,image=64,dictLabels=dictLabels,cmap='gray')

# <b><div style='padding:18px;background-color:#d8dcd6;color:#030aa7;font-size:130%; border-radius:12px 12px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>Liste des échantillons de données</div></b>

In [ ]:
listeEchantillons = {
    'pathmnist':{
                  0:"adipose",
                  1:"background",
                  2:"debris",
                  3:"lymphocytes",
                  4:"mucus",
                  5:"smooth\nmuscle",
                  6:"normal\ncolon mucosa",
                  7:"cancer-associated\nstroma",
                  8:"colorectal\nadenocarcinoma\nepithelium"
                 },
    'chestmnist':{
                  0: "atelectasis",
                  1: "cardiomegaly",
                  2: "effusion",
                  3: "infiltration",
                  4: "mass",
                  5: "nodule",
                  6: "pneumonia",
                  7: "pneumothorax",
                  8: "consolidation",
                  9: "edema",
                  10: "emphysema",
                  11: "fibrosis",
                  12: "pleural",
                  13: "hernia"
                 },
    'dermamnist':{
                  0:"actinic keratoses\nand intraepithelial carcinoma",
                  1:"basal cell carcinoma",
                  2:"benign keratosis-like lesions",
                  3:"dermatofibroma",
                  4:"melanoma",
                  5:"melanocytic nevi",
                  6:"vascular lesions"
                  },
    'octmnist':{
                  0: "choroidal\nneovascularization",
                  1: "diabetic\nmacular\nedema",
                  2: "drusen",
                  3: "normal"
                  },
    'pneumoniamnist':{
                  0: "normal",
                  1: "pneumonia"
                  },
    'retinamnist':{
                  0:"0",
                  1:"1",
                  2:"2",
                  3:"3",
                  4:"4"
                  },
    'breastmnist':{
                  0: "malignant",
                  1: "normal, benign"
                  },
    'bloodmnist':{
                  0:"basophil",
                  1:"eosinophil",
                  2:"erythroblast",
                  3:"immature",# granulocytes(myelocytes, metamyelocytes and promyelocytes)",
                  4:"lymphocyte",
                  5:"monocyte",
                  6:"neutrophil",
                  7:"platelet"
                  },
    'tissuemnist':{
                  0:"Collecting Duct\nConnecting Tubule",
                  1:"Distal Convoluted Tubule",
                  2:"Glomerular\nendothelial cells",
                  3:"Interstitial\nendothelial cells",
                  4:"Leukocytes",
                  5:"Podocytes",
                  6:"Proximal\nTubule Segments",
                  7:"Thick\nAscending Limb"
                  },
    'organamnist':{
                  0 : "bladder",
                  1 : "femur-left",
                  2 : "femur-right",
                  3 : "heart",
                  4 : "kidney-left",
                  5 : "kidney-right",
                  6 : "liver",
                  7 : "lung-left",
                  8 : "lung-right",
                  9 : "pancreas",
                  10 : "spleen"
                  }
}

# <b><div style='padding:18px;background-color:#d8dcd6;color:#030aa7;font-size:130%; border-radius:12px 12px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>Création des fichiers TFRecors</div></b>

In [ ]:
def creationTFRecords(nom,echantillon,donnees,repertoireSauvegardes):
    def _bytes_feature(value):
        if isinstance(value, type(tf.constant(0))): value = value.numpy()  
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
    def _float_feature(value):
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    
    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    
    def _formatageEnregistrement(image, label):
        if len(image.shape) == 2 : image = np.expand_dims(image, axis=-1)
        image = tf.image.encode_jpeg(image)
        formatTFRecord = {
                          'image': _bytes_feature(image),
                          'label': _int64_feature(label)
                          }
        return tf.train.Example(features=tf.train.Features(feature=formatTFRecord)).SerializeToString()
    
    nomEchantillon = {"apprentissage":"train",
                      "validation":"val",
                      "test":"test"}
        
    fichierSharde = tf.io.TFRecordWriter(os.path.join(repertoireSauvegardes,f'{nom}-{echantillon}.tfrecords'))
    for i, (image, label) in enumerate(zip(donnees[f'{nomEchantillon[echantillon]}_images'], 
                                           donnees[f'{nomEchantillon[echantillon]}_labels'])):  
        fichierSharde.write(_formatageEnregistrement(image.astype("uint8"), label[0]))

    fichierSharde.close() 

In [ ]:
for nom in listeEchantillons:
    print(nom)
    donnees = np.load(f"../donnees/MedMNIST(Standardized Biomedical Image)/{nom}.npz")
    creationTFRecords(nom,'apprentissage',donnees,repertoireSauvegardes)
    creationTFRecords(nom,'validation',donnees,repertoireSauvegardes)
    creationTFRecords(nom,'test',donnees,repertoireSauvegardes)    


In [ ]:
repertoireSauvegardes

In [ ]:
for nom in ['pathmnist_224',
            'octmnist_224',
            'bloodmnist_224',
            'tissuemnist_224',
            'organamnist_224']:
    print(nom)
    donnees = np.load(f"../donnees/MedMNIST(Standardized Biomedical Image)/{nom}.npz")
    creationTFRecords(nom,'apprentissage',donnees,repertoireSauvegardes)
    creationTFRecords(nom,'validation',donnees,repertoireSauvegardes)
    creationTFRecords(nom,'test',donnees,repertoireSauvegardes)    


# <b><div style='padding:18px;background-color:#d8dcd6;color:#030aa7;font-size:130%; border-radius:12px 12px; box-shadow: 8px 8px 8px #042b4c;text-align: left'>Contrôle des fichiers TFRecors</div></b>

In [ ]:
def controleFichiersTfRecords(nom,dictLabels):
    def _traitementImage(image):
        image = tf.cast(tf.image.decode_jpeg(image, channels=3), tf.int32)
        return image
    
    def _lectureTFRecord(enregistrement):
        formatTFRecord = ({
            'image': tf.io.FixedLenFeature([], tf.string),
            'label':  tf.io.FixedLenFeature([], tf.int64),
        })
    
        enregistrement = tf.io.parse_single_example(enregistrement, formatTFRecord)
        enregistrement['image'] =  _traitementImage(enregistrement['image'])
        return enregistrement
    
    def _dictToImageLabel(enregistrement):
        # return enregistrement['image'],tf.one_hot(enregistrement['label'], 5)
        return enregistrement['image'],enregistrement['label']
    
    pipeline_train = tf.data.TFRecordDataset(os.path.join(repertoireSauvegardes,f'{nom}-apprentissage.tfrecords'))
    pipeline_train = pipeline_train.map(_lectureTFRecord)
    pipeline_train = pipeline_train.map(_dictToImageLabel)
    pipeline_train = pipeline_train.batch(25)
    pipeline_train = pipeline_train.prefetch(tf.data.AUTOTUNE)  
    
    image_batch, label_batch = next(iter(pipeline_train))
    
    plt.figure(figsize=(24, 24))
    for n in range(25):
        ax = plt.subplot(5, 5, n + 1)
        plt.imshow(image_batch[n])
        plt.title(dictLabels[label_batch[n].numpy()])
        plt.axis("off")

In [ ]:
for nom in listeEchantillons:
    controleFichiersTfRecords(nom,listeEchantillons[nom])